In [ ]:
# This the code for scrapping https://dailytrust.com/
# Daily trust uses javascript to load more news, thus the need for selenium to dynamically extract the contents from the pages. 
# As a result, you need google chrome driver 
# https://chromedriver.storage.googleapis.com/index.html?path=90.0.4430.24/

In [1]:
!pip install bs4 selenium

In [2]:
import requests
import re
from bs4 import BeautifulSoup
from urllib import request
from urllib.request import Request, urlopen
import time
agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}

In [3]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome("C:/Users/alabi/Downloads/chromedriver_win32 /chromedriver.exe", chrome_options=options)

C:\Users\alabi\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [4]:
#scrap the category links from the navigation bar/pane
def scrap_navlinks(url):
    page_request = request.Request(url, headers=agent)
    page = request.urlopen(page_request)
    soup = BeautifulSoup(page, 'html.parser')
    #The daily sun news website has a navigation bar with the CSS class attribute containing the texts below
    xp = soup.find("nav", {"class": "menu_menu__2zVZ6"}).findAll('a')
    navlinks = list(set(['https://dailytrust.com'+i['href'] for i in xp if '/topics/' in i['href'] and 'music' not in i['href'] and 'videos' not in i['href'] and 'column' not in i['href'] and 'podcast' not in i['href']]))
    return navlinks

In [5]:
def scrap_link(catgory, click_more = 50):
    driver.get(catgory)
    more_buttons = driver.find_elements_by_class_name("more_button__2MXuW")
    for x in range(click_more):#len(more_buttons)):
        if more_buttons[0].is_displayed():
            driver.execute_script("arguments[0].click();", more_buttons[0])
            time.sleep(1)
    page_source = driver.page_source    
    urls = []
    links = []
    soup = BeautifulSoup(page_source, 'lxml')
    res = soup.find_all('div',{'class':'list_card__MiAg3'})
    alllinks = [x.find_all('a',{'aria-label':re.compile("^\w+")}) for x in res]
    links.extend(alllinks)
    mergelinks = [j for i in links for j in i]
    urls.extend(list(set([t['href'] for t in mergelinks])))
    return urls

In [6]:
#scrap the articles with links in the `urls` variable
def getSoup(links):
  print("getting soup objects for all the links ... ")
  soupx = []
  cnt=0
  for url in links: 
    page_request = request.Request(url, headers=agent)
    page = request.urlopen(page_request)
    #response = requests.get(page_url)
    #print(page.getcode())
    soup = BeautifulSoup(page, 'html.parser')
    soupx.append(soup)
    cnt = cnt + 1
    progresse = (cnt/len(links) * 100) 
    if progresse  % 10 == 0:
      print ("Got ", progresse, "% of soup objects")
  print("got the soup object for all the links ... ")
  return soupx
#get the news content from the soup objects extracted above (return the title, date and texts)
def getcontent(soupx):
  print("scrapping the articles ... ")
  cnt=0;
  title=[];time=[]; texts=[]
  for soups in soupx:
    cnt=cnt+1
    if soups.find("h1", {"class":"typography_h1v1__2lx3I"})!= None:
      title.append(soups.find("h1", {"class":"typography_h1v1__2lx3I"}).text.replace("\t","").replace("\n",""))
    else:
      title.append("")
    if soups.find("div", {"class":"Page_time__3YjtX"})!= None:
      time.append(soups.find("div", {"class":"Page_time__3YjtX"}).text.replace("\t","").replace("\n",""))
    else:
      time.append("")
  
    if soups.find("article", {"class":"typography_body__32m2U"})!=None:
      result = soups.find("article", {"class":"typography_body__32m2U"}).findAll('p')
      txtstring=""
      for x in result:
      #print (x.text)
        txtstring+=x.text.replace(u'\xa0', u' ').replace('\n'," ")+" \n"
      texts.append(txtstring.strip())
    else:
      texts.append("") 

    progresse = (cnt/len(soupx) * 100) 
    if progresse  % 10 == 0:
      print ("Scrapped ", progresse, "% of the articles")

  return title,time,texts

In [7]:
#scrap the category links from the navigation pane
navlinks = scrap_navlinks('https://dailytrust.com/')
print('Got ',len(navlinks), ' categories from the navigation bar.')
print(navlinks)
urls = []
for i in range(len(navlinks)):
    print(navlinks[i])
    #you can play around with click_more to get more news
    urls.extend(scrap_link(navlinks[i]), click_more = 50)
    
urls = [scrap_link(navlinks[i]) for i in range(len(navlinks))] #returns a list of lists
#flatten the list of lists, that is urls
urls = set(["https://dailytrust.com"+y for x in urls for y in x])

Got  8  categories from the navigation bar.
['https://dailytrust.com/topics/business', 'https://dailytrust.com/topics/opinion', 'https://dailytrust.com/topics/crime', 'https://dailytrust.com/topics/politics', 'https://dailytrust.com/topics/agriculture', 'https://dailytrust.com/topics/sports', 'https://dailytrust.com/topics/news', 'https://dailytrust.com/topics/international']
https://dailytrust.com/topics/business
https://dailytrust.com/topics/opinion
https://dailytrust.com/topics/crime
https://dailytrust.com/topics/politics
https://dailytrust.com/topics/agriculture
https://dailytrust.com/topics/sports
https://dailytrust.com/topics/news
https://dailytrust.com/topics/international


In [8]:
soups = getSoup(list(urls))
title,time,texts = getcontent(soups)
#create a dictionarty
import pandas as pd
d = {'Date':time,'Title':title,'Text':texts}
df = pd.DataFrame(d)

getting soup objects for all the links ... 
Got  10.0 % of soup objects
Got  20.0 % of soup objects
Got  30.0 % of soup objects
Got  40.0 % of soup objects
Got  50.0 % of soup objects
Got  60.0 % of soup objects
Got  70.0 % of soup objects
Got  80.0 % of soup objects
Got  90.0 % of soup objects
Got  100.0 % of soup objects
got the soup object for all the links ... 
scrapping the articles ... 
Scrapped  10.0 % of the articles
Scrapped  20.0 % of the articles
Scrapped  30.0 % of the articles
Scrapped  40.0 % of the articles
Scrapped  50.0 % of the articles
Scrapped  60.0 % of the articles
Scrapped  70.0 % of the articles
Scrapped  80.0 % of the articles
Scrapped  90.0 % of the articles
Scrapped  100.0 % of the articles


In [9]:
df

,Date,Title,Text
0,Fri Apr 02 2021,Top seed Medvedev follows Osaka out of Miami Open,Top-seeded Daniil Medvedev crashed out of the ...
1,Wed May 26 2021,"Lekki seaport to create 2,000 direct jobs — MD",The Managing Director of Lekki Port LFTZ Enter...
2,Fri Apr 09 2021,Man excretes N423m cocaine at Lagos airport,"A Madrid, Spain-bound suspected drug trafficke..."
3,Mon Sep 07 2020,"Egypt discovers 2,500-year-old coffins intact",Egypt has announced the discovery of a collect...
4,Thu Jan 21 2021,America after Trump: End of an error?,"Today, the world awakes with a sigh. A ‘bigly’..."
5,Thu Apr 29 2021,Osinbajo lauds Value4Dairy Consortium and plan...,Vice President Yemi Osinbajo has welcome the s...
6,Sat May 01 2021,Enyimba draw Pyramids in Confederation Cup qua...,"Former African champions, Enyimba Internationa..."
7,Mon Feb 08 2021,"Zamfara APC factions resolve crisis, pick Yari...",The crisis rocking the All Progressives Congr...
8,Tue May 25 2021,Bayelsa judicial panel awards N36m to victims ...,The Bayelsa State Panel of Inquiry on Police B...
9,Fri Mar 12 2021,"Ex-Speaker Bankole meets Buni, Badaru ahead de...","A former Speaker of House of Representatives, ..."


In [8]:
'''def scrap_link(catgory, click_more = 50):
driver.get("https://dailytrust.com/topics/business")
more_buttons = driver.find_elements_by_class_name("more_button__2MXuW")
for x in range(len(more_buttons)):
  if more_buttons[x].is_displayed():
      driver.execute_script("arguments[0].click();", more_buttons[x])
      time.sleep(1)
page_source = driver.page_source'''

IndexError: list index out of range